#  Final Mini Project

In [3]:
import numpy as np
import cv2 as cv
from collections import deque

In [4]:
#default called trackbar function 
def setValues(x):
    pass
    #print("")


# Creating the trackbars needed for adjusting the Track bar value for colour adjustment
cv.namedWindow("Color detectors")
cv.createTrackbar("Upper Hue", "Color detectors", 153, 255,setValues)
cv.createTrackbar("Upper Saturation", "Color detectors", 255, 255,setValues)
cv.createTrackbar("Upper Value", "Color detectors", 255, 255,setValues)
cv.createTrackbar("Lower Hue", "Color detectors", 64, 255,setValues)
cv.createTrackbar("Lower Saturation", "Color detectors", 72, 255,setValues)
cv.createTrackbar("Lower Value", "Color detectors", 49, 255,setValues)

# Giving different arrays to handle colour points of different colour
b_points = [deque(maxlen=1024)]
g_points = [deque(maxlen=1024)]
r_points = [deque(maxlen=1024)]
y_points = [deque(maxlen=1024)]

# These indexes will be used to mark the points in particular arrays of specific colour
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

# Kernel is square or any shape which we want to apply on the image. Is is used for blurring, sharpening, embossing , edge detection and more.
# Kernel tells you how to change the value of any given pixel by combining it with different amounts of neighbouring pixels.
kernel = np.ones((5,5),np.uint8)

# list of colors
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
colorIndex = 0        # default 0 for blue color

# Here is code for Canvas setup
paintWindow = np.zeros((471,636,3)) + 255
paintWindow = cv.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
paintWindow = cv.rectangle(paintWindow, (160,1), (255,65), colors[0], -1)
paintWindow = cv.rectangle(paintWindow, (275,1), (370,65), colors[1], -1)
paintWindow = cv.rectangle(paintWindow, (390,1), (485,65), colors[2], -1)
paintWindow = cv.rectangle(paintWindow, (505,1), (600,65), colors[3], -1)

# stting font
font = cv.FONT_HERSHEY_SIMPLEX

cv.putText(paintWindow, "CLEAR", (49, 33), font, 0.5, (0, 0, 0), 2, cv.LINE_AA)
cv.putText(paintWindow, "BLUE", (185, 33), font, 0.5, (255, 255, 255), 2, cv.LINE_AA)
cv.putText(paintWindow, "GREEN", (298, 33), font, 0.5, (255, 255, 255), 2, cv.LINE_AA)
cv.putText(paintWindow, "RED", (420, 33), font, 0.5, (255, 255, 255), 2, cv.LINE_AA)
cv.putText(paintWindow, "YELLOW", (520, 33), font, 0.5, (150,150,150), 2, cv.LINE_AA)
cv.namedWindow('Paint', cv.WINDOW_AUTOSIZE)


# Taking video from webcam
cap = cv.VideoCapture(0)

# to continue capturing frames
while True:
    
    # Reading the frame from the camera
    ret, frame = cap.read()
    
    # Flipping the frame to see same side of yours
    frame = cv.flip(frame, 1)
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    
    # taking trackbar values  
    u_hue = cv.getTrackbarPos("Upper Hue", "Color detectors")
    u_saturation = cv.getTrackbarPos("Upper Saturation", "Color detectors")
    u_value = cv.getTrackbarPos("Upper Value", "Color detectors")
    l_hue = cv.getTrackbarPos("Lower Hue", "Color detectors")
    l_saturation = cv.getTrackbarPos("Lower Saturation", "Color detectors")
    l_value = cv.getTrackbarPos("Lower Value", "Color detectors")
    # storing these trackbar values in an array
    Upper_hsv = np.array([u_hue,u_saturation,u_value])
    Lower_hsv = np.array([l_hue,l_saturation,l_value])


    # Adding the colour buttons 
    frame = cv.rectangle(frame, (40,1), (140,65), (122,122,122), -1)
    frame = cv.rectangle(frame, (160,1), (255,65), colors[0], -1)
    frame = cv.rectangle(frame, (275,1), (370,65), colors[1], -1)
    frame = cv.rectangle(frame, (390,1), (485,65), colors[2], -1)
    frame = cv.rectangle(frame, (505,1), (600,65), colors[3], -1)
    cv.putText(frame, "CLEAR ALL", (49, 33), font, 0.5, (255, 255, 255), 2, cv.LINE_AA)
    cv.putText(frame, "BLUE", (185, 33), font, 0.5, (255, 255, 255), 2, cv.LINE_AA)
    cv.putText(frame, "GREEN", (298, 33), font, 0.5, (255, 255, 255), 2, cv.LINE_AA)
    cv.putText(frame, "RED", (420, 33), font, 0.5, (255, 255, 255), 2, cv.LINE_AA)
    cv.putText(frame, "YELLOW", (520, 33), font, 0.5, (150,150,150), 2, cv.LINE_AA)


    # thresholding the hsv image to get only the pointer    
    Mask = cv.inRange(hsv, Lower_hsv, Upper_hsv)
    
    # Erosion: The thickness or size of the foreground object decreases or simply the white region
    # decreases in the image.
    # It is useful for removing small white noises.
    Mask = cv.erode(Mask, kernel, iterations=1)
    
    # Opening: It is erosion followed by dilation
    Mask = cv.morphologyEx(Mask, cv.MORPH_OPEN, kernel)
    
    # Dilation :It increases the white region in the image or the size of the foreground object increases.
    Mask = cv.dilate(Mask, kernel, iterations=1)

    # Find contours for the pointer after idetifying it
    cnts,_ = cv.findContours(Mask.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    center = None

    # If the contours are formed
    if len(cnts) > 0:
        # sorting the contours to find biggest 
        cnt = sorted(cnts, key = cv.contourArea, reverse = True)[0]
        
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv.minEnclosingCircle(cnt)
        
        # Draw the circle around the contour
        cv.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        
        # Calculating the center of the detected contour
        M = cv.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        # Now checking if the user wants to click on any button above the screen 
        if center[1] <= 65:
            if 40 <= center[0] <= 140:         # Clear Button
                b_points = [deque(maxlen=512)]
                g_points = [deque(maxlen=512)]
                r_points = [deque(maxlen=512)]
                y_points = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                paintWindow[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                    colorIndex = 0            # Blue
            elif 275 <= center[0] <= 370:
                    colorIndex = 1            # Green
            elif 390 <= center[0] <= 485:
                    colorIndex = 2            # Red
            elif 505 <= center[0] <= 600:
                    colorIndex = 3            # Yellow
        else :
            if colorIndex == 0:
                b_points[blue_index].appendleft(center)
            elif colorIndex == 1:
                g_points[green_index].appendleft(center)
            elif colorIndex == 2:
                r_points[red_index].appendleft(center)
            elif colorIndex == 3:
                y_points[yellow_index].appendleft(center)
                
    # Append the next deques when nothing is detected to avoid messing up
    else:
        b_points.append(deque(maxlen=512))
        blue_index += 1
        g_points.append(deque(maxlen=512))
        green_index += 1
        r_points.append(deque(maxlen=512))
        red_index += 1
        y_points.append(deque(maxlen=512))
        yellow_index += 1

        
    # Draw lines of all the colors on the canvas and frame 
    points = [b_points, g_points, r_points, y_points]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                
                cv.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    # Show all the windows
    cv.imshow("Tracking", frame)
    cv.imshow("Paint", paintWindow)
    cv.imshow("mask",Mask)

    # If the 'q' key is pressed then stop the application 
    if cv.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and all resources
cap.release()
cv.destroyAllWindows()